# Cyclamenのデータ結合

# インストール

In [43]:
!pip install scipy
!pip install xlrd
!pip install openpyxl
!pip install matplotlib
!pip install numpy

In [48]:
!pip show numpy

Name: numpy
Version: 1.15.4
Summary: NumPy: array processing for numbers, strings, records, and objects.
Home-page: http://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: None
License: BSD
Location: /work/firstuser/anaconda3/envs/beakerx/lib/python3.7/site-packages
Requires: 
Required-by: scipy, pandas, matplotlib


# ライブラリのインポート

In [49]:
import numpy as np
import scipy as sp
import pandas as pd
from pandas import Series, DataFrame

#エクセル読み込み・書き込み
import xlrd
import pprint
import openpyxl

# 可視化ライブラリ
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

# 小数第3位まで表示
%precision 3

RuntimeError: module compiled against API version 0xc but this version of numpy is 0xa

ImportError: numpy.core.multiarray failed to import

# ファイル読み込み

In [42]:
#入力データファイルパス
ref_filename='C:/Users/SST170320\Desktop\Work_Cyclamen/Ref対応表.xlsx'
result_filename='C:/Users/SST170320/Desktop/Work_Cyclamen/解析結果アノテーション.xlsx'

# データセット

In [ ]:
#データフレーム作成

#
def preprocess(x):
    # 不要な行, 列のフィルタなど、データサイズを削減する処理＊何かする場合はここに
    return x

#指定フォルダcsvを読み込み、chunksize = 50にしてメモリから溢れないように読み込み
def read_csv_to_dataframe(result_filename):
    dataset1= pd.read_excel(result_filename, sheet_name=0,chunksize=50)
    df0_result = pd.concat((preprocess(r) for r in dataset1), ignore_index=True)#preprocessを行いながら、読み込み
    return df0_result

#解析結果アノテーションのシートごとに読み込み
#dataset1= pd.read_excel('C:/Users/SST170320/Desktop/Work_Cyclamen/解析結果アノテーション.xlsx', sheet_name=0,chunksize=50)
#df0_result_table = pd.concat((preprocess(r) for r in dataset1), ignore_index=True)#preprocessを行いながら、読み込み
#df1_result_table= pd.read_excel('C:/Users/SST170320/Desktop/Work_Cyclamen/解析結果アノテーション.xlsx', sheet_name=1)
#df2_result_table= pd.read_excel('C:/Users/SST170320/Desktop/Work_Cyclamen/解析結果アノテーション.xlsx', sheet_name=2)
#df3_result_table= pd.read_excel('C:/Users/SST170320/Desktop/Work_Cyclamen/解析結果アノテーション.xlsx', sheet_name=3)


#ファイルの中身確認
print(df_ref_compare.head())
print(df0_result.head())

# データフレームの編集

In [ ]:
#最初の行削除
df0_result_trim = df0_result.drop(0)

#列名変更（keyとなる列名をそろえる
df0_ref_trim = df.rename(columns={'sseqid(SNmp_SNp_Trinity)': 'key'})
df0_result_trim = df.rename(columns={'コンティグ名': 'key'})

#結合
df_output = pd.merge(df0_result_table,df_ref_compare,how='left')


#エクセルへ書き込み
df_output.to_excel('data/dst/pandas_to_excel.xlsx', sheet_name='new_sheet_name')
